# Merging Boolean Networks

## Overview

This Jupyter notebook provides tools for merging multiple Boolean network models using  `OR`, `AND`, and `Inhibitor Wins`. The notebook includes functions for:

- **Parsing Boolean Expressions:** Extracting activators and inhibitors from Boolean expressions.
- **Merging Boolean Networks:** Combining Boolean network models using different methods (`OR`, `AND`, and `Inhibitor Wins`).
- **Writing Merged Networks to File:** Saving the final merged network to a text file or SBML-qual file.

## Usage

1. **Input format:**
   - Text file similar to the Python package `Boolnet`, as described [here](https://rdrr.io/cran/BoolNet/man/loadNetwork.html)
   - SBML-qual format
1. **Define Input Networks:** Provide the Boolean networks as dictionaries where keys are gene names and values are Boolean expressions.
2. **Merge Networks:** Use the `merge_networks` function to merge the networks using the desired method.
3. **Save to File:** Use the `write_network_to_file` function to save the merged network.
   - Text file similar to input
   - SBML-qual format


In [1]:
model1name = "Bonzanni2013"
model2name = "Krumsiek2011"

In [2]:
# Need to install libsbml in Colomoto environment
#!conda install -c conda-forge python-libsbml

In [3]:
import re
import libsbml
import os
import boolean
from rpy2.robjects.packages import importr
boolnet = importr("BoolNet")

In [4]:
def read_network(file_path, body_separator=","):
    """
    Reads a Boolean network model from a text file or an SBML-qual file and returns a dictionary representation.
    The Boolean expressions are cleaned of unnecessary parentheses.
    
    :param file_path: Path to the model file (text file or SBML-qual file).
    :param body_separator: Character used to separate targets and factors in the text file.
    :return: Dictionary with gene names as keys and their cleaned Boolean expressions as values.
    """
    # Determine the file type based on the extension
    file_extension = os.path.splitext(file_path)[1].lower()
    
    if file_extension in [".txt", ".csv"]:
        return read_text_network(file_path, body_separator)
    elif file_extension in [".xml", ".sbml"]:
        return read_sbml_qual_network(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a .txt, .csv, .xml, or .sbml file.")

def read_text_network(file_path, body_separator=","):
    """
    Reads a Boolean network model from a text file and returns a dictionary representation.
    The Boolean expressions are cleaned of unnecessary parentheses.
    
    :param file_path: Path to the model file.
    :param body_separator: Character used to separate targets and factors in the file.
    :return: Dictionary with gene names as keys and their cleaned Boolean expressions as values.
    """
    network = {}

    with open(file_path, 'r') as file:
        lines = file.readlines()
        
    # Strip comments and empty lines
    lines = [re.sub(r'#.*', '', line).strip() for line in lines if line.strip() and not line.startswith("#")]
    
    # Check if the file is empty after stripping comments and empty lines
    if not lines:
        raise ValueError("The file is empty or contains only comments.")
    
    # Check and parse the header
    header = [part.strip() for part in lines[0].split(body_separator)]
    if len(header) < 2 or header[0].lower() != "targets" or header[1].lower() not in ["functions", "factors"]:
        raise ValueError(f"Invalid header: {lines[0]}")
    
    # Process each rule
    for line in lines[1:]:
        # Split the line based on the body_separator, but only at the first occurrence
        parts = [part.strip() for part in line.split(body_separator, 1)]
        if len(parts) < 2:
            raise ValueError(f"Invalid rule format: {line}")
        
        target = parts[0].strip()
        expression = parts[1].strip()
        
        # Validate gene names
        if not re.match(r'^[a-zA-Z_][a-zA-Z0-9_]*$', target):
            raise ValueError(f"Invalid gene name: {target}")
        
        network[target] = simplify_expression(expression)
    
    return network

def read_sbml_qual_network(file_path):
    """
    Reads a Boolean network model from an SBML-qual file and returns a dictionary representation.
    The Boolean expressions are cleaned of unnecessary parentheses.
    
    :param file_path: Path to the SBML-qual file.
    :return: Dictionary with gene names as keys and their cleaned Boolean expressions as values.
    """
    reader = libsbml.SBMLReader()
    document = reader.readSBML(file_path)
    model = document.getModel()

    # Parse qualitative species (genes)
    genes = {}
    listOfSpecies = model.getPlugin("qual").getListOfQualitativeSpecies()

    for i in range(listOfSpecies.size()):
        species = listOfSpecies.get(i)
        gene_name = species.getId()
        genes[gene_name] = species.getName() or gene_name

    # Parse transitions (interactions)
    network = {}
    listOfTransitions = model.getPlugin("qual").getListOfTransitions()

    for i in range(listOfTransitions.size()):
        transition = listOfTransitions.get(i)
        output_list = transition.getListOfOutputs()
        function_terms = transition.getListOfFunctionTerms()

        # Get function terms and convert to Boolean expressions
        expression = []
        for j in range(function_terms.size()):
            function_term = function_terms.get(j)
            math = function_term.getMath()
            expression.append(parse_mathml(math, genes))

        if len(expression) > 1:
            expression = " | ".join(expression)
        else:
            expression = expression[0]

        for output in output_list:
            gene = genes[output.getQualitativeSpecies()]
            network[gene] = simplify_expression(expression)

    return network


def parse_mathml(math, genes):
    """
    Parses MathML expressions to Boolean expressions.
    
    :param math: MathML object from libSBML.
    :param genes: Dictionary of gene names.
    :return: Boolean expression as a string.
    """
    name = math.getType()
    
    if name == libsbml.AST_CONSTANT_TRUE:
        return "1"
    elif name == libsbml.AST_CONSTANT_FALSE:
        return "0"
    elif name == libsbml.AST_NAME:
        return genes.get(math.getName(), math.getName())
    elif name == libsbml.AST_REAL:
        return str(math.getReal())
    elif name == libsbml.AST_INTEGER:
        return str(math.getInteger())
    elif name == libsbml.AST_RELATIONAL_EQ:
        left_child = parse_mathml(math.getChild(0), genes)
        right_child = parse_mathml(math.getChild(1), genes)
        if right_child == "1":
            return left_child  # GATA1 == 1 should return GATA1
        elif right_child == "0":
            return f"!{left_child}"  # GATA1 == 0 should return !GATA1
        else:
            return f"({left_child} == {right_child})"
    elif name == libsbml.AST_LOGICAL_AND:
        children = [parse_mathml(math.getChild(i), genes) for i in range(math.getNumChildren())]
        return f"({' & '.join(children)})"
    elif name == libsbml.AST_LOGICAL_OR:
        children = [parse_mathml(math.getChild(i), genes) for i in range(math.getNumChildren())]
        return f"({' | '.join(children)})"
    elif name == libsbml.AST_LOGICAL_NOT:
        child = parse_mathml(math.getChild(0), genes)
        if math.getChild(0).getType() in (libsbml.AST_LOGICAL_AND, libsbml.AST_LOGICAL_OR):
            return f"!({child})"
        else:
            return f"!{child}"
    else:
        raise ValueError(f"Unsupported MathML element: {libsbml.formulaToString(math)}")


def simplify_expression(expression):
    """
    Simplifies the Boolean expression by removing redundant parentheses using boolean.py package.
    
    :param expression: The Boolean expression as a string.
    :return: The simplified expression.
    """
    algebra = boolean.BooleanAlgebra()
    algebra.parse(expression)
    simplified_expression = str(algebra.parse(expression).simplify())
    # Replace '~' with '!'
    simplified_expression = simplified_expression.replace('~', '!')

    return simplified_expression

In [5]:
expression = '!SPI1&(FLI1|GATA1|GATA2)'
algebra = boolean.BooleanAlgebra()
algebra.parse(expression)
simplified_expression = str(algebra.parse(expression).simplify())

In [6]:
simplified_expression

'~SPI1&(FLI1|GATA1|GATA2)'

In [7]:
# Example usage:
network1 = read_network("../Models/" + model1name + ".txt")
network2 = read_network("../Models/" + model2name + ".sbml")
print("Network #1:")
print(network1)
print("Network #2:")
print(network2)

Network #1:
{'SPI1': '(FLI1|RUNX1|SPI1)&(!GATA1|!SPI1)', 'GATA1': 'GATA1&!SPI1&TAL1', 'GATA2': '(ERG|FLI1|(GATA2&TAL1))&(!GATA1|!ZFPM1)&(!GATA2|!HHEX)', 'TAL1': 'ERG|FLI1|(GATA1&TAL1)|(GATA2&TAL1)', 'ZFPM1': 'GATA2&TAL1', 'FLI1': '!GATA1&(ERG|FLI1|GATA2)', 'ERG': '(!CBFA2T3|!TAL1)&(ERG|FLI1|GATA2|RUNX1)', 'CBFA2T3': 'GATA2&TAL1', 'RUNX1': '(ERG|FLI1|RUNX1|SPI1|(GATA2&TAL1))&(!RUNX1|!SMAD6)', 'SMAD6': 'ERG|FLI1|(GATA2&TAL1)', 'HHEX': 'ERG|FLI1|(GATA2&TAL1)'}
Network #2:
{'GATA2': 'GATA2&!SPI1&(!GATA1|!ZFPM1)', 'GATA1': '!SPI1&(FLI1|GATA1|GATA2)', 'ZFPM1': 'GATA1', 'KLF1': '!FLI1&GATA1', 'FLI1': 'GATA1&!KLF1', 'TAL1': 'GATA1&!SPI1', 'CEBPA': 'CEBPA&(!GATA1|!TAL1|!ZFPM1)', 'SPI1': '!GATA1&!GATA2&(CEBPA|SPI1)', 'JUN': '!GFI1&SPI1', 'EGR1': '!GFI1&JUN&SPI1', 'EGR2': '!GFI1&JUN&SPI1', 'NAB1': '!GFI1&JUN&SPI1', 'GFI1': 'CEBPA&!EGR1&!EGR2&!NAB1'}


In [8]:
def parse_expression(expression):
    """
    Parses a Boolean expression to separate activators and inhibitors.
    """
    activators = []
    inhibitors = []

    # Detect complex inhibitors like !(GATA1 & GATA2)
    complex_inhibitors = re.findall(r'!\(([^)]+)\)', expression)
    for complex_inhibitor in complex_inhibitors:
        inhibitors.append(complex_inhibitor.strip())
        expression = expression.replace(f"!({complex_inhibitor.strip()})", '')

    # Detect simple inhibitors like !GATA1
    simple_inhibitors = re.findall(r'![a-zA-Z_][a-zA-Z0-9_]*', expression)
    inhibitors.extend([inhibitor[1:] for inhibitor in simple_inhibitors])

    # Remove all inhibitors from the expression to identify activators
    for inhibitor in inhibitors:
        expression = expression.replace(f"&!{inhibitor}", '').replace(f"|!{inhibitor}", '').replace(f"!{inhibitor}", '')

    # Detect activator groups connected by '&' or within parentheses
    activator_groups = re.findall(r'\([^()]*\)|[^()]*&[^()]*', expression)
    # Remove the symbol & behind the last activator name and filter out empty strings and exactly '(|)'
    activator_groups = [group.rstrip('&').strip() for group in activator_groups if group.strip() != '(|)']
    
    for group in activator_groups:
        if group.strip():
            activators.append(group.strip())
        expression = expression.replace(group, '')

    # Detect remaining simple activators
    activators.extend(re.findall(r'[a-zA-Z_][a-zA-Z0-9_]*', expression))
    # Remove empty parentheses from the activators list
    activators = [activator for activator in activators if activator != '()']

    return activators, inhibitors

def check_inhibitor_wins_rule(expression):
    """
    Ensures that the "Inhibitor Wins" rule is respected within a single expression.
    """
    components = re.split(r'\|', expression)
    
    for component in components:
        component = component.strip()
        if '&' in component:
            activators, inhibitors = parse_expression(component)
            if activators and inhibitors:
                continue
        if "!" not in component:
            continue
        activators, inhibitors = parse_expression(component)
        if activators and inhibitors:
            return False
    
    return True

def merge_networks(networks, method="OR", descriptive=True):
    """
    Merges multiple Boolean network models using the specified method (OR, AND, or Inhibitor Wins).
    Outputs a description of the merged model if descriptive is True.
    """
    if method not in ["OR", "AND", "Inhibitor Wins"]:
        raise ValueError("Invalid method. Use 'OR', 'AND', or 'Inhibitor Wins'.")

    merged_network = {}
    overlap_genes = set()
    descriptions = {}
    warning_gene = {}

    # Track the number of genes in each individual model
    individual_gene_counts = [len(network) for network in networks]

    # Iterate through each network
    for idx, network in enumerate(networks):
        for gene, expression in network.items():
            if gene in merged_network:
                overlap_genes.add(gene)
                
                # Determine how to merge the expressions based on the method
                if method == "OR":
                    merged_expression = f"({merged_network[gene]})|({expression})"
                elif method == "AND":
                    merged_expression = f"({merged_network[gene]})&({expression})"
                elif method == "Inhibitor Wins":
                    activators, inhibitors = parse_expression(expression)
                    existing_activators, existing_inhibitors = parse_expression(merged_network[gene])
                    combined_activators = list(set(existing_activators + activators))
                    combined_inhibitors = list(set(existing_inhibitors + inhibitors))
                    
                    activator_expr = '|'.join(filter(None, combined_activators)) if combined_activators else ""
                    inhibitor_expr = '|'.join(filter(None, combined_inhibitors)) if combined_inhibitors else ""
                    
                    activator_genes = set(re.split(r'\|', activator_expr))
                    inhibitor_genes = set(re.split(r'\|', inhibitor_expr))
                    if activator_genes & inhibitor_genes:
                        overlappings = list(activator_genes & inhibitor_genes)
                        for overlapping in overlappings:
                            activator_expr = re.sub(rf'(\||&)?{overlapping}(\||&)?', '', activator_expr).strip('|&')
                        warning_gene[gene] = overlappings
                    
                    if inhibitor_expr:
                        if '&' in inhibitor_expr or '|' in inhibitor_expr:
                            merged_expression = f"({activator_expr})&!({inhibitor_expr})"
                        else:
                            merged_expression = f"({activator_expr})&!{inhibitor_expr}"
                    else:
                        merged_expression = activator_expr
                
                else:
                    merged_expression = expression
                
                if descriptive:
                    descriptions.setdefault(gene, []).append((f"Model {idx + 1}", expression))
                    descriptions[gene].append(("Merged", merged_expression))
                
                # Simplify the merged expression
                merged_network[gene] = simplify_expression(merged_expression)
                
            else:
                merged_network[gene] = expression
                if descriptive:
                    descriptions[gene] = [(f"Model {idx + 1}", expression)]

    if descriptive:
        print(f"Merging Method: {method}")
        print(f"Total Genes in Merged Network: {len(merged_network)}")
        print(f"Number of Genes in Each Individual Model:")
        for i, count in enumerate(individual_gene_counts, 1):
            print(f"  Model {i}: {count} genes")
        print(f"Overlapping Genes: {len(overlap_genes)}")
        if overlap_genes:
            print(f"Overlapping Genes List: {', '.join(overlap_genes)}")
            for gene in overlap_genes:
                print(f"\nGene: {gene}")
                if gene in warning_gene.keys():
                    print(f"Warning: possible conflicts for {warning_gene[gene]}, keeping only inhibitor.")
                for desc in descriptions[gene]:
                    print(f"  {desc[0]} Function: {desc[1]}")
        else:
            print("No overlapping genes found.")
     
    return merged_network

In [9]:
# Example usage:

# Merge using OR approach
merged_network_or = merge_networks([network1, network2], method="OR")

Merging Method: OR
Total Genes in Merged Network: 18
Number of Genes in Each Individual Model:
  Model 1: 11 genes
  Model 2: 13 genes
Overlapping Genes: 6
Overlapping Genes List: FLI1, SPI1, TAL1, GATA1, ZFPM1, GATA2

Gene: FLI1
  Model 1 Function: !GATA1&(ERG|FLI1|GATA2)
  Model 2 Function: GATA1&!KLF1
  Merged Function: (!GATA1&(ERG|FLI1|GATA2))|(GATA1&!KLF1)

Gene: SPI1
  Model 1 Function: (FLI1|RUNX1|SPI1)&(!GATA1|!SPI1)
  Model 2 Function: !GATA1&!GATA2&(CEBPA|SPI1)
  Merged Function: ((FLI1|RUNX1|SPI1)&(!GATA1|!SPI1))|(!GATA1&!GATA2&(CEBPA|SPI1))

Gene: TAL1
  Model 1 Function: ERG|FLI1|(GATA1&TAL1)|(GATA2&TAL1)
  Model 2 Function: GATA1&!SPI1
  Merged Function: (ERG|FLI1|(GATA1&TAL1)|(GATA2&TAL1))|(GATA1&!SPI1)

Gene: GATA1
  Model 1 Function: GATA1&!SPI1&TAL1
  Model 2 Function: !SPI1&(FLI1|GATA1|GATA2)
  Merged Function: (GATA1&!SPI1&TAL1)|(!SPI1&(FLI1|GATA1|GATA2))

Gene: ZFPM1
  Model 1 Function: GATA2&TAL1
  Model 2 Function: GATA1
  Merged Function: (GATA2&TAL1)|(GATA1)



In [10]:
# Merge using AND approach
merged_network_and = merge_networks([network1, network2], method="AND")

Merging Method: AND
Total Genes in Merged Network: 18
Number of Genes in Each Individual Model:
  Model 1: 11 genes
  Model 2: 13 genes
Overlapping Genes: 6
Overlapping Genes List: FLI1, SPI1, TAL1, GATA1, ZFPM1, GATA2

Gene: FLI1
  Model 1 Function: !GATA1&(ERG|FLI1|GATA2)
  Model 2 Function: GATA1&!KLF1
  Merged Function: (!GATA1&(ERG|FLI1|GATA2))&(GATA1&!KLF1)

Gene: SPI1
  Model 1 Function: (FLI1|RUNX1|SPI1)&(!GATA1|!SPI1)
  Model 2 Function: !GATA1&!GATA2&(CEBPA|SPI1)
  Merged Function: ((FLI1|RUNX1|SPI1)&(!GATA1|!SPI1))&(!GATA1&!GATA2&(CEBPA|SPI1))

Gene: TAL1
  Model 1 Function: ERG|FLI1|(GATA1&TAL1)|(GATA2&TAL1)
  Model 2 Function: GATA1&!SPI1
  Merged Function: (ERG|FLI1|(GATA1&TAL1)|(GATA2&TAL1))&(GATA1&!SPI1)

Gene: GATA1
  Model 1 Function: GATA1&!SPI1&TAL1
  Model 2 Function: !SPI1&(FLI1|GATA1|GATA2)
  Merged Function: (GATA1&!SPI1&TAL1)&(!SPI1&(FLI1|GATA1|GATA2))

Gene: ZFPM1
  Model 1 Function: GATA2&TAL1
  Model 2 Function: GATA1
  Merged Function: (GATA2&TAL1)&(GATA1)


In [11]:
# Merge using Inhibitor Wins approach
merged_network_inhibitor_wins = merge_networks([network1, network2], method="Inhibitor Wins")

Merging Method: Inhibitor Wins
Total Genes in Merged Network: 18
Number of Genes in Each Individual Model:
  Model 1: 11 genes
  Model 2: 13 genes
Overlapping Genes: 6
Overlapping Genes List: FLI1, SPI1, TAL1, GATA1, ZFPM1, GATA2

Gene: FLI1
  Model 1 Function: !GATA1&(ERG|FLI1|GATA2)
  Model 2 Function: GATA1&!KLF1
  Merged Function: ((ERG|FLI1|GATA2))&!(GATA1|KLF1)

Gene: SPI1
  Model 1 Function: (FLI1|RUNX1|SPI1)&(!GATA1|!SPI1)
  Model 2 Function: !GATA1&!GATA2&(CEBPA|SPI1)
  Merged Function: ((CEBPA|SPI1)|(FLI1|RUNX1|SPI1))&!(GATA1|GATA2|SPI1)

Gene: TAL1
  Model 1 Function: ERG|FLI1|(GATA1&TAL1)|(GATA2&TAL1)
  Model 2 Function: GATA1&!SPI1
  Merged Function: (FLI1|ERG|(GATA2&TAL1)|GATA1|(GATA1&TAL1))&!SPI1

Gene: GATA1
  Model 1 Function: GATA1&!SPI1&TAL1
  Model 2 Function: !SPI1&(FLI1|GATA1|GATA2)
  Merged Function: (GATA1&TAL1|(FLI1|GATA1|GATA2))&!SPI1

Gene: ZFPM1
  Model 1 Function: GATA2&TAL1
  Model 2 Function: GATA1
  Merged Function: GATA1|GATA2&TAL1

Gene: GATA2
  Model 

In [12]:
def write_network_to_file(network, filename, format="text"):
    """
    Writes the merged Boolean network to a file in the specified format.
    
    :param network: Dictionary representing the merged Boolean network.
    :param filename: The name of the file to write the network to.
    :param format: The format of the output file, either "text" or "sbml".
    """
    if format == "text":
        with open(filename + ".txt", 'w') as file:
            file.write("targets, factors\n")
            for gene, expression in network.items():
                file.write(f"{gene}, {expression}\n")
        print(f"Network successfully written to {filename}.txt")
    
    elif format == "sbml":
        # Write to a temporary text file
        temp_text_file = "temp_network.txt"
        with open(temp_text_file, 'w') as file:
            file.write("targets, factors\n")
            for gene, expression in network.items():
                file.write(f"{gene}, {expression}\n")
        
        # Load the network from the text file using boolnet
        net = boolnet.loadNetwork(temp_text_file)
        
        # Export to SBML format
        boolnet.toSBML(net, filename + ".sbml")
        
        # Delete the temporary text file
        os.remove(temp_text_file)
        
        print(f"Network successfully written to {filename}.sbml")
    
    else:
        raise ValueError("Invalid format specified. Use 'text' or 'sbml'.")

In [15]:
merged_and_name = "merged_and_" + model1name + "_" + model2name
merged_inhibitor_wins_name = "merged_inhibitor_wins_" + model1name + "_" + model2name
merged_or_name = "merged_or_" + model1name + "_" + model2name

write_network_to_file(merged_network_and, "../Models/" + merged_and_name, format="text")
write_network_to_file(merged_network_inhibitor_wins, "../Models/" + merged_inhibitor_wins_name, format="text")
write_network_to_file(merged_network_or, "../Models/" + merged_or_name, format="text")
write_network_to_file(merged_network_and, "../Models/" + merged_and_name, format="sbml")
write_network_to_file(merged_network_inhibitor_wins, "../Models/" + merged_inhibitor_wins_name, format="sbml")
write_network_to_file(merged_network_or, "../Models/" + merged_or_name, format="sbml")

Network successfully written to ../Models/merged_and_Bonzanni2013_Krumsiek2011.txt
Network successfully written to ../Models/merged_inhibitor_wins_Bonzanni2013_Krumsiek2011.txt
Network successfully written to ../Models/merged_or_Bonzanni2013_Krumsiek2011.txt
Network successfully written to ../Models/merged_and_Bonzanni2013_Krumsiek2011.sbml
Network successfully written to ../Models/merged_inhibitor_wins_Bonzanni2013_Krumsiek2011.sbml
Network successfully written to ../Models/merged_or_Bonzanni2013_Krumsiek2011.sbml
